# <center>Перенос стиля с помощью архитектуры *Adaptive Instance Normalization*</center>
Сделаем необходимые импорты и инициализируем константы

In [1]:
import os
# Less tensorflow backend logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import keras
import tensorflow as tf
from net.style_autoencoder import StyleTransfer
IMAGES_FOLDER = '/mnt/s/CV/StyleTransferData/'
CONTENT_FOLDER = os.path.join(IMAGES_FOLDER, 'test2015')
STYLE_FOLDER = os.path.join(IMAGES_FOLDER, 'wikiart')
BATCH_SIZE = 8
EPOCHS = 5
IMG_SIZE = (256, 256)
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Напишем функцию для удаления повреждённых файлов (необходимо раскомментировать последние 2 строки ячейки, если появляются проблемы с чтением файлов во время обучения)

In [2]:
def delete_corrupted_imgs(path) -> None:
    def remove(file) -> None:
        print('=='*10)
        os.remove(os.path.join(path, file))
        print(f'Deleted corrupted {file}')
        
    files = os.listdir(path)
    for file in files:
        try:
            # Throws exception or returns None if image file corrupted
            img = tf.image.decode_jpeg(tf.io.read_file(os.path.join(path, file)))
            if img is None:
                remove(file)
        except Exception as e:
            remove(file)

# delete_corrupted_imgs(STYLE_FOLDER)
# delete_corrupted_imgs(CONTENT_FOLDER)

Инициализируем 2 датасета: `content_dataset` - для изображений контента и `style_dataset` - для стилей, объединим их с помощью `tf.data.Dataset.zip` в один датасет.

In [3]:
dataset_config = dict(
    label_mode=None, 
    labels=None,
    shuffle=True,
    image_size=IMG_SIZE,
    batch_size=None,
    crop_to_aspect_ratio=True
)

content_dataset = keras.utils.image_dataset_from_directory(
    CONTENT_FOLDER,
    **dataset_config
)
style_dataset = keras.utils.image_dataset_from_directory(
    STYLE_FOLDER,
    **dataset_config
)

dataset = tf.data.Dataset.zip((content_dataset, style_dataset)).batch(BATCH_SIZE, drop_remainder=True)

Found 81434 files belonging to 1 classes.
Found 81434 files belonging to 1 classes.


[<b>Ссылка на оригинальную статью <i>AdaIN</i></b>](https://arxiv.org/pdf/1703.06868.pdf)

[<b>Моя реализация данной модели на <i>Keras / Tensorflow</i></b>](net/style_autoencoder.py)

В отличии от оригинального решения, я использовал вес функции потерь стиля $\lambda = 10$ вместо $\lambda = 0.01$, чтобы получить "больше стиля" в результирующем изображении

In [6]:
model = StyleTransfer()
optimizer = keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.InverseTimeDecay(
        initial_learning_rate=1e-4,
        # Reduce lr every 10 steps
        decay_steps=10,
        decay_rate=5e-5
    )
)
loss_fn = keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss_fn=loss_fn)
# 2 - (content, style), None's - (B, H, W), 3 - channels
model.build(input_shape=(2, None, None, None, 3))
model.summary()

Model: "style_transfer_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ada_in_2 (AdaIN)            multiple                  0         
                                                                 
 encoder_2 (Encoder)         multiple                  3505728   
                                                                 
 decoder_2 (Decoder)         multiple                  3505219   
                                                                 
Total params: 7010955 (26.74 MB)
Trainable params: 3505219 (13.37 MB)
Non-trainable params: 3505736 (13.37 MB)
_________________________________________________________________


Обучаем модель

In [7]:
history = model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[
        # Save model after every epoch without rewrite
        keras.callbacks.ModelCheckpoint(
            './checkpoint/style_model_{epoch}.keras',
            monitor='total_loss',
            save_best_only=False,
            save_freq='epoch'
        )
    ]
)

Epoch 1/5


I0000 00:00:1710407899.975024    8033 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10179/10179 [==============================] - 1375s 134ms/step - style_loss: 439565.4062 - content_loss: 336777.1875 - total_loss: 776342.6875 - learning_rate: 9.7539e-05
Epoch 2/5
10179/10179 [==============================] - 1451s 142ms/step - style_loss: 265485.1875 - content_loss: 245536.0156 - total_loss: 511021.6875 - learning_rate: 9.2925e-05
Epoch 3/5
10179/10179 [==============================] - 1382s 135ms/step - style_loss: 237794.5000 - content_loss: 222680.7656 - total_loss: 460476.7188 - learning_rate: 8.8728e-05
Epoch 4/5
10179/10179 [==============================] - 1372s 135ms/step - style_loss: 222228.3438 - content_loss: 210117.9219 - total_loss: 432345.8125 - learning_rate: 8.4893e-05
Epoch 5/5
10179/10179 [==============================] - 1370s 134ms/step - style_loss: 211999.8594 - content_loss: 201801.8594 - total_loss: 413802.5312 - learning_rate: 8.1377e-05


Результаты обучения будут в [<b>другом ноутбуке</b>](InferenceTest.ipynb)